In [1]:
Distance = [[0,1064,673,1401,277],
            [1064,0,958,1934,377],
            [673,957,0,1001,399],
            [1401,1934,1001,0,387],
            [277,377,399,387,0]]
W_1 = {1:A,2:E,3:D

MP2
===
You are the owner of a smart manufacturing company, with factories in five different cities: Atlanta, Boston, Chicago, Denver, and Edmonton. Each factory makes a different type of component: for convenience, let's call the components A, B, C, D, and E. The locations of your factories are a bit inconvenient to one another; the following table gives the distance (in miles) from one factory to another.

Distance | A | B | C | D | E
---------|---|---|---|---|---
A| 0 | 1064| 673 | 1401 | 277
B|1064| 0|	958|	1934|	337
C|673|958|0|	    1001|	    399
D|1401| 1934| 1001	|0|387
E|277|	337|	399|	387|	0
            
---
From those components, you manufacture five different types of widgets. Each widget is composed of five widgets of different types, as follows:

* Widget 1 = Components AEDCA
* Widget 2 = Components BEACD
* Widget 3 = Components BABCE
* Widget 4 = Components DADBD
* Widget 5 = Components BECBD